<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1ddc978d87cdd52e8c8f2b58b191914824b8da132e08b41d6ba90f7b46b235f7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-13 11:23:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -11.10 K (-0.08%)
Current PnL: -28.39 L (-18.11%)
CY Booked + Current PnL: -14.10 L (-8.99%)
-------------------
Total profit:  1.71 L
Total loss:  -30.10 L
-------------------
Total Booked + Current PnL: 12.88 L (9.92%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.16%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.85 L (66.71%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.46,16.23,4.44,21.40,13698.0,43080.0,308506.0,7.85,66.0,X-LC,6.14,14.0,3.14,2.19,23.58,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.35,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,70.0,L-LC,8.33,182.0,0.45,1.13,28.48,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.11,-22.24,28.72,0.09,22506.0,-22413.0,78364.0,81.86,34.0,M-SC,3.46,253.0,-1.00,0.56,2.95,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.86,-3.11,25.96,22.05,24645.0,-3045.0,94935.0,-16.44,48.0,H-SC,5.77,168.0,-0.12,0.67,37.99,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.09,-14.86,17.47,0.02,25965.0,-25933.0,148626.0,126.56,52.0,M-SC,8.58,240.0,-1.00,1.06,29.28,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,4.40,-31.26,181.67,93.63,126090.0,-31557.0,69406.0,-26.39,51.0,H-SC,16.73,143.0,-0.25,0.49,4.93,SR,ATH,CHEMICALS
55,RAJESHEXPO,518.00,2.76,-67.58,208.70,0.07,92803.0,-92710.0,44467.0,1530.05,36.0,L-SC,1.49,269.0,-1.00,0.32,10.59,OX40N,NTT,JEWELLERY
66,SIS,528.00,2.66,-23.36,58.23,21.26,49550.0,-25938.0,85094.0,2008.81,51.0,H-SC,4.71,163.0,-0.52,0.60,14.97,OX40N,NTT,MISC
26,GILLETTE,11206.78,2.40,-5.12,39.56,32.42,95300.0,-12996.0,240900.0,-21.96,45.0,X-SC,12.32,87.0,-0.14,1.71,9.21,X40,ATH,FMCG
13,BSOFT,831.70,1.89,-23.65,97.48,50.79,104687.0,-33260.0,107393.0,-2.20,48.0,H-SC,10.28,171.0,-0.32,0.76,25.04,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-2.56,-20.00,64.07,31.26,118289.0,-46144.0,184624.0,-62.63,29.0,X-MC,20.05,56.0,-0.39,1.31,0.00,X40N,ATH,IT
0,5PAISA,593.00,-2.19,-31.27,64.27,12.90,96518.0,-68320.0,150176.0,121.15,56.0,H-SC,9.71,173.0,-0.71,1.07,24.87,OX40N,NTT,FINANCE
31,HAVELLS,2069.16,-1.34,-8.38,44.60,32.48,134027.0,-27481.0,300510.0,-13.06,47.0,X-MC,5.06,69.0,-0.21,2.14,2.96,X40,ATH,ELECTRICAL
18,COFFEEDAY,80.00,-1.29,-43.15,137.95,35.27,89047.0,-48999.0,64550.0,-55.73,41.0,L-SC,4.67,268.0,-0.55,0.46,57.99,XR,NTT,HOTELS
45,JIOFIN,387.00,-1.21,-8.41,35.91,24.48,87938.0,-22480.0,244885.0,-9.94,36.0,X-LC,1.63,37.0,-0.26,1.74,41.94,XY24,BTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.86,-0.57,21.43,20.74,43292.0,-1155.0,202015.0,31.15,54.0,M-MC,5.48,189.0,-0.03,1.44,9.99,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.30,-3.87,116.28,107.91,165040.0,-5711.0,141933.0,-21.77,46.0,M-SC,10.07,220.0,-0.03,1.01,5.83,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.09,-14.86,17.47,0.02,25965.0,-25933.0,148626.0,126.56,52.0,M-SC,8.58,240.0,-1.00,1.06,29.28,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.82,-17.28,59.60,32.03,65000.0,-22775.0,109060.0,-10.50,66.0,H-SC,6.71,157.0,-0.35,0.78,22.16,OX40N,NTT,IT
47,KANSAINER,340.00,1.56,-21.83,45.16,13.47,95200.0,-58860.0,210807.0,-67.92,50.0,H-SC,4.32,159.0,-0.62,1.50,7.19,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,0.11,-22.24,28.72,0.09,22506.0,-22413.0,78364.0,81.86,34.0,M-SC,3.46,253.0,-1.00,0.56,2.95,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.35,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,70.0,L-LC,8.33,182.0,0.45,1.13,28.48,XY25,ATH,MINING
84,WIPRO,420.00,0.11,8.03,59.45,72.27,108831.0,13615.0,183063.0,-7.44,52.0,M-LC,6.28,101.0,0.13,1.30,14.71,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.11,8.03,59.45,72.27,108831.0,13615.0,183063.0,-7.44,52.0,M-LC,6.28,101.0,0.13,1.30,14.71,XR,NTT,IT
25,FINCABLES,1641.55,0.30,-3.87,116.28,107.91,165040.0,-5711.0,141933.0,-21.77,46.0,M-SC,10.07,220.0,-0.03,1.01,5.83,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.33,-7.14,122.61,106.72,140429.0,-8803.0,114533.0,-54.06,38.0,H-SC,7.80,138.0,-0.06,0.81,14.57,AR,ATH,MISC
37,IEX,219.00,-0.51,-4.11,54.99,48.63,105440.0,-8210.0,191744.0,-36.24,51.0,H-SC,13.19,137.0,-0.08,1.36,7.04,XR,NTT,MISC
49,MASFIN,398.61,0.86,-3.11,25.96,22.05,24645.0,-3045.0,94935.0,-16.44,48.0,H-SC,5.77,168.0,-0.12,0.67,37.99,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.71,-17.65,34.52,10.78,68316.0,-42414.0,197904.0,-50.96,13.0,X-LC,7.89,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
7,AWL,485.00,0.86,-28.14,118.75,57.19,278287.0,-91780.0,234347.0,-66.05,25.0,X-MC,9.96,58.0,-0.33,1.67,0.86,XY24,NTT,FMCG
52,PAGEIND,51605.08,0.23,-12.82,49.82,30.62,102963.0,-30384.0,206670.0,-38.08,28.0,X-MC,16.85,57.0,-0.30,1.47,0.48,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.16,-41.22,130.00,35.20,98329.0,-53032.0,75638.0,-0.79,28.0,X-SC,17.50,93.0,-0.54,0.54,0.16,X40,NTT,FOOTWEAR
78,UNITDSPR,1644.00,-0.84,-3.30,24.89,20.77,56356.0,-7723.0,226421.0,-11.68,28.0,X-MC,1.60,70.0,-0.14,1.61,3.39,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.84,-3.30,24.89,20.77,56356.0,-7723.0,226421.0,-11.68,28.0,X-MC,1.60,70.0,-0.14,1.61,3.39,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-1.21,-8.41,35.91,24.48,87938.0,-22480.0,244885.0,-9.94,36.0,X-LC,1.63,37.0,-0.26,1.74,41.94,XY24,BTT,FINANCE
65,SIEMENS,4671.50,0.46,-19.54,56.00,25.51,83846.0,-36370.0,149725.0,-8.15,38.0,H-LC,2.13,50.0,-0.43,1.06,10.34,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.72,1.71,41.28,43.70,104158.0,4253.0,252321.0,-6.68,62.0,X-MC,2.16,73.0,0.04,1.79,17.02,XY24,BTT,FMCG
73,TCS,4389.97,0.47,-10.51,34.88,20.71,118065.0,-39748.0,338489.0,-22.98,53.0,X-LC,2.27,3.0,-0.34,2.41,13.10,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-2.56,-20.00,64.07,31.26,118289.0,-46144.0,184624.0,-62.63,29.0,X-MC,20.05,56.0,-0.39,1.31,0.00,X40N,ATH,IT
43,ITC,452.00,-0.71,-17.65,34.52,10.78,68316.0,-42414.0,197904.0,-50.96,13.0,X-LC,7.89,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
11,BATAINDIA,2096.00,0.16,-41.22,130.00,35.20,98329.0,-53032.0,75638.0,-0.79,28.0,X-SC,17.50,93.0,-0.54,0.54,0.16,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.06,-28.23,128.72,64.15,219828.0,-67171.0,170780.0,-58.06,34.0,X-SC,6.50,84.0,-0.31,1.21,0.27,XY24,BTT,CEMENT
58,RELAXO,1176.00,0.42,-48.78,197.38,52.31,146742.0,-70815.0,74345.0,-45.50,38.0,X-SC,7.83,92.0,-0.48,0.53,0.42,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.25,-28.82,100.67,42.84,46583.0,-18733.0,46273.0,-53.17,47.0,X-SC,25.98,83.0,-0.40,0.33,4.69,XY24,NTT,MISC
58,RELAXO,1176.00,0.42,-48.78,197.38,52.31,146742.0,-70815.0,74345.0,-45.50,38.0,X-SC,7.83,92.0,-0.48,0.53,0.42,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.16,-41.22,130.00,35.20,98329.0,-53032.0,75638.0,-0.79,28.0,X-SC,17.50,93.0,-0.54,0.54,0.16,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.28,-0.04,24.13,24.08,30619.0,-57.0,126893.0,-8.06,50.0,X-SC,5.90,91.0,-0.00,0.90,19.56,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.72,-17.96,74.70,43.32,99814.0,-29252.0,133620.0,-29.54,43.0,X-SC,8.07,90.0,-0.29,0.95,2.60,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.71,-17.65,34.52,10.78,68316.0,-42414.0,197904.0,-50.96,13.0,X-LC,7.89,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.83,-28.75,72.59,22.98,116842.0,-64936.0,160962.0,-25.52,36.0,X-LC,3.84,2.0,-0.56,1.14,1.24,XY24,NTT,AUTO
73,TCS,4389.97,0.47,-10.51,34.88,20.71,118065.0,-39748.0,338489.0,-22.98,53.0,X-LC,2.27,3.0,-0.34,2.41,13.10,X40,ATH,IT
81,VBL,671.64,1.09,0.84,34.54,35.67,110006.0,2648.0,318490.0,-11.08,59.0,X-LC,2.40,5.0,0.02,2.26,14.81,X40N,ATH,FMCG
41,INFY,1972.00,-0.31,8.31,23.95,34.24,81162.0,25995.0,338883.0,-15.65,45.0,X-LC,7.55,6.0,0.32,2.41,17.28,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.09,-33.21,98.97,32.89,51034.0,-25642.0,51565.0,-736.93,64.0,L-MC,8.16,259.0,-0.50,0.37,42.01,XR,NTT,BANKS
13,BSOFT,831.70,1.89,-23.65,97.48,50.79,104687.0,-33260.0,107393.0,-2.20,48.0,H-SC,10.28,171.0,-0.32,0.76,25.04,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.09,-14.86,17.47,0.02,25965.0,-25933.0,148626.0,126.56,52.0,M-SC,8.58,240.0,-1.00,1.06,29.28,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.35,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,70.0,L-LC,8.33,182.0,0.45,1.13,28.48,XY25,ATH,MINING
84,WIPRO,420.00,0.11,8.03,59.45,72.27,108831.0,13615.0,183063.0,-7.44,52.0,M-LC,6.28,101.0,0.13,1.30,14.71,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-2.19,-31.27,64.27,12.90,96518.0,-68320.0,150176.0,121.15,56.0,H-SC,9.71,173.0,-0.71,1.07,24.87,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,0.82,-17.28,59.60,32.03,65000.0,-22775.0,109060.0,-10.50,66.0,H-SC,6.71,157.0,-0.35,0.78,22.16,OX40N,NTT,IT
17,COALINDIA,484.83,-0.35,5.94,12.54,19.23,19988.0,8936.0,159396.0,24.27,70.0,L-LC,8.33,182.0,0.45,1.13,28.48,XY25,ATH,MINING
57,RECLTD,446.00,-0.86,-0.57,21.43,20.74,43292.0,-1155.0,202015.0,31.15,54.0,M-MC,5.48,189.0,-0.03,1.44,9.99,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,-0.09,-33.21,98.97,32.89,51034.0,-25642.0,51565.0,-736.93,64.0,L-MC,8.16,259.0,-0.50,0.37,42.01,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.47
2,50,76.52


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.59
MC    30.73
LC    26.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.24
X40      24.03
X40N     12.76
XY25     10.86
XR        8.96
AR        8.54
OX40N     7.65
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.33
X-LC    23.19
H-SC    22.77
M-SC    11.22
X-SC     7.82
H-MC     4.59
M-MC     1.44
M-LC     1.30
L-LC     1.13
H-LC     1.06
L-SC     0.78
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.28,-8.27,43.79
IT,12.89,-17.98,77.49
FINANCE,11.00,-15.23,63.59
MISC,7.01,-31.60,85.43
ELECTRICAL,5.86,-13.83,55.52
PAINTS,5.84,-12.49,29.20
INSURANCE,4.72,-2.02,36.93
PHARMA,3.98,-5.15,38.25
AUTO,2.77,-34.49,81.11


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3277460.0,21
AR,1370614.0,10
XR,1231486.0,12
X40,1141489.0,15
X40N,946435.0,9
OX40N,717711.0,10
XY25,414453.0,7
SR,285164.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3589051.0,24
X-MC,1612221.0,16
M-SC,1482435.0,15
X-LC,959890.0,13
X-SC,818869.0,8
H-MC,433505.0,3
L-SC,181850.0,2
M-LC,108831.0,1
H-LC,83846.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1268614.0      6
           AR         928259.0      5
M-SC       XY24       850087.0      6
H-SC       XR         793332.0      7
X-MC       X40        525432.0      7
           XY24       450073.0      3
X-LC       X40        422428.0      6
X-MC       X40N       414382.0      4
X-SC       X40N       328210.0      3
H-SC       OX40N      313682.0      4
M-SC       OX40N      311226.0      5
X-SC       XY24       297030.0      3
H-SC       SR         285164.0      2
H-MC       AR         226629.0      2
X-MC       XY25       222334.0      2
H-MC       XY24       206876.0      1
X-LC       XY24       204780.0      2
           X40N       203843.0      2
X-SC       X40        193629.0      2
M-SC       XR         189242.0      2
           AR         131880.0      2
X-LC       XY25       128839.0      3
M-LC       XR         108831.0      1
L-SC       OX40N       92803.0      1
           XR          89047.0      1
H-LC       AR          83846.0      1
L-MC       XR          51034.0      1
M-MC       XY25        43292.0      1
L-LC       XY25        19988.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
